<a href="https://colab.research.google.com/github/cbsobral/ml-fies/blob/main/Module02c_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 02c - Random Forest

In this module, we perform the following steps:

1. Load the data from Mod_00 and create sets and targets for train and test datasets;
2. Create down and up samples from the original train set;
3. Perform grid search for all of them;
4. Calculate performance measures. 


*Note: Due to the size of our data set and the technical conditions available, the time to perform cross-validation for the random forest model was considerable long. For example, a randomized CV with 3 parameters sampled and 4 folds took up to 7h. As we know, good quality results increase with larger number of parameters sampled. Due to time limitation, we were only able to use cross-validation with a reduced set of hyperparameters candidates. Some configurations were tested individually when cross-validation was not an option. Nevertheless, the final model improved in relation to the intial one.*

### 1 - Load Data

Here, we import the training and testing sets created in Module00_Data. 


In [ ]:
import pandas as pd

url_train = "https://drive.google.com/file/d/1IP7jyXkLgD_Ouy5cL6fJk4VUA5qRB2PK/view?usp=sharing"
path_train = "https://drive.google.com/uc?export=download&id="+url_train.split("/")[-2]
train = pd.read_csv(path_train)
train.shape

(351001, 31)

In [ ]:
url_test = "https://drive.google.com/file/d/1v4FqKwt7NzG5RM6d9f1y7CLIdKq69jSS/view?usp=sharing"
path_test = "https://drive.google.com/uc?export=download&id="+url_test.split("/")[-2]
test = pd.read_csv(path_test)
test.shape

(87751, 31)

In [ ]:
train_set = train.drop("default", axis=1) # drop targets for training set
train_target = train["default"].copy()

In [ ]:
test_set = test.drop("default", axis=1) # drop targets for test set
test_target = test["default"].copy()

### 2 - Pipeline

The pipeline contains functions that will be used to transform the dataset. For the numeric attributes, the stardardization is performed by the StandardScaler. For ordinal attributes, variables are encoded by the OrdinalEncoder, and for categorical, theOneHotEncoder. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([("num_imputer", SimpleImputer(strategy="median")),
                         ("std_scaler", StandardScaler()), 
                         ])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

ord_pipeline = Pipeline([("ord_imputer", SimpleImputer(strategy="most_frequent")),
                         ("ord_encoder", OrdinalEncoder()),
                         ])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline(steps=[('one_hot', OneHotEncoder())])

In [ ]:
ord_attribs = ["igc","date_contract"] # 2 attributes

num_attribs = ["family_income",   #17
               "personal_income",
               "high_school_endyear",
               "n_sem_course",
               "n_completed_sem",
               "sem_funded",
               "fam_size",
               "income_pc",
               "tuition_current",
               "inc_prop",
               "perc_requested",
               "loan_value_sem",
               "student_resource",
               "loan_value",
               "loan_limit",
               "total_debt",
               "age"]
  

cat_attribs = ["semester_enroll",  #9
               "gender",
               "occupation", 
               "marital_status",
               "ethnicity", 
               "public_hs", 
               "state_course", 
               "degree", 
               "contract_phase"]

In [ ]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
        ("ord", ord_pipeline, ord_attribs)
        ])

In [ ]:
train_prepared = full_pipeline.fit_transform(train_set)
train_prepared[:1]

<1x94 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Row format>

In [ ]:
test_prepared = full_pipeline.fit_transform(test_set)
test_prepared[:1]

<1x94 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Row format>

### 3 - Fine-Tuning


Randomized search on hyper parameters:

In [ ]:
# Time to run: +- 7h

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Criterion
criterion = ['gini', 'entropy']

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

rf = RandomForestClassifier(random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 3, cv = 4, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(train_prepared, train_target)

Best parameters:
{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 200}

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'bootstrap': [False],
    'max_depth': [40, 60],
    'max_features': ["auto"],
    'min_samples_leaf': [5],
    'min_samples_split': [8],
    'n_estimators': [150, 300]}


rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(rf,param_grid=param_grid,cv=3)
grid_search.fit(train_prepared, train_target)
grid_search.cv_results_['params']
grid_search.cv_results_['mean_test_score']

Best parameters
{'bootstrap': False,
 'max_depth': 60,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'n_estimators': 300}


In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 50, max_features = 'auto', min_samples_leaf= 4, 
                            min_samples_split = 5, n_estimators = 200, random_state=42)
rf.fit(train_prepared, train_target)

Result 0.790198064967537 AUC

In [ ]:
 rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 60, max_features = 'auto', min_samples_leaf= 5, 
                              min_samples_split = 8, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

Result 0.7896261537831428 AUC

In [ ]:
rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 50, max_features = 'auto', min_samples_leaf= 4, 
                            min_samples_split = 5, n_estimators = 250, random_state=42)
rf.fit(train_prepared, train_target)

Result 0.7905031207466537 AUC

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 50, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=50,
                       min_samples_leaf=4, min_samples_split=5,
                       n_estimators=300, random_state=42)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7905609772221769

In [ ]:
loss_rf

0.18172957094162634

 Result 0.7905609772221769 AUC

Testing if ExtraTreesClassifier() looks promising:

In [ ]:
rf_extra = ExtraTreesClassifier()
rf_extra.fit(train_prepared, train_target)

Result: 0.7464396156070683 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = True, criterion = 'entropy', max_depth = 50, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42, class_weight='balanced')
rf.fit(train_prepared, train_target)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf


0.7862866965536888

In [ ]:
loss_rf

0.18599798529566028

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(criterion = 'entropy', max_depth = 50, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
brf.fit(train_prepared, train_target)

BalancedRandomForestClassifier(criterion='entropy', max_depth=50,
                               min_samples_leaf=4, min_samples_split=5,
                               n_estimators=300, random_state=42)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_brf = brf.predict_proba(test_prepared)

auc_brf = roc_auc_score(test_target, pred_brf[:,1])
loss_brf = brier_score_loss(test_target, pred_brf[:,1])

auc_brf

0.7852046606840751

In [ ]:
loss_brf

0.1921610807966289

In [ ]:
#Best model for the original data:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.790575936333656

In [ ]:
loss_rf

0.1817306326853902

Saving the best model for the original data:

In [ ]:
import joblib

filename = 'BestRF.sav'
joblib.dump(rf, filename, compress=3)


['BestRF.sav']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 60, max_features = 'auto', min_samples_leaf= 5, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=60, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7896261537831428

0.7896261537831428

In [ ]:
#Testing Bootstrap:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = True, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7867739634598249

In [ ]:
#Testing min_sample_leaf = 5:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 5, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)
rf.fit(train_prepared, train_target)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7896865466877714

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

rus = RandomUnderSampler(random_state=42)
data_under, target_under = rus.fit_resample(train_prepared, train_target)

ros = RandomOverSampler(random_state=42)
data_over, target_over = ros.fit_resample(train_prepared, train_target)

In [ ]:
#Testing Best Model with Undersampling:

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)

rf.fit(data_under, target_under)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7894275007426411

In [ ]:


from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Criterion
criterion = ['gini', 'entropy']

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion': criterion}

rf = RandomForestClassifier(random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 3, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(data_under, target_under)

rf_random.best_params_

In [ ]:
#Testing Best Model with Undersampling (after Random search): it took 9 hours to run because of high n_estimators

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 30, max_features = 'auto', min_samples_leaf= 2, 
                      min_samples_split = 2, n_estimators = 1000, random_state=42)

rf.fit(data_under, target_under)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7916932434176123

In [ ]:
#Best Model with Undersampling with reasonable n_estimators that allows it to be trained in less than 3 hours

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 30, max_features = 'auto', min_samples_leaf= 2, 
                      min_samples_split = 2, n_estimators = 300, random_state=42)

rf.fit(data_under, target_under)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf_under = roc_auc_score(test_target, pred_rf[:,1])
loss_rf_under = brier_score_loss(test_target, pred_rf[:,1])

print(auc_rf_under, loss_rf_under)

In [ ]:
print(auc_rf, loss_rf)

In [ ]:
#Testing Best Model with Oversampling 

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)

rf.fit(data_over, target_over)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=55, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = rf.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])

auc_rf

0.7911611214789033

Saving the best model for oversampling:

In [ ]:
import joblib
joblib.dump(rf, "RF_over.joblib", compress=3)

['RF_over.joblib']

### 4 - Performance Evaluation

In [ ]:
import joblib

RF_over = joblib.load('/RF_over.joblib')
RF = joblib.load('/BestRF.sav')

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf = RF.predict_proba(test_prepared)

auc_rf = roc_auc_score(test_target, pred_rf[:,1])
loss_rf = brier_score_loss(test_target, pred_rf[:,1])


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

pred_rf_over = RF_over.predict_proba(test_prepared)

auc_rf_over = roc_auc_score(test_target, pred_rf_over[:,1])
loss_rf_over = brier_score_loss(test_target, pred_rf_over[:,1])

In [ ]:
auc_rf = 0.790576
auc_rf_over = 0.791161
auc_rf_under = 0.790106

loss_rf = 0.181731
loss_rf_over = 0.182321
loss_rf_under = 0.189422

# List with AUC scores
auc_list = [auc_rf, auc_rf_over, auc_rf_under]

# List with Brier Scores
bs_list = [loss_rf, loss_rf_over, loss_rf_under]

# List with model names
names_list = ['Original','Oversample', 'Undersample']

# Dataframe 
auc_df = pd.DataFrame({"Sample": names_list, "AUC": auc_list, "BS": bs_list})
auc_df.sort_values(by = "AUC", ascending=False)

,Model,AUC,BS
1,Oversampling,0.791161,0.182321
0,Original,0.790576,0.181731
2,Undersampling,0.790106,0.189422


Conclusion:

The best Random forest model is as follows bellow trained in an oversampled data set:

rf = RandomForestClassifier( bootstrap = False, criterion = 'entropy', max_depth = 55, max_features = 'auto', min_samples_leaf= 4, 
                      min_samples_split = 5, n_estimators = 300, random_state=42)